# Homework 7: Stock Prediction

## GE Stock Prediction
Here we are going to predict GE stock prices. Example from https://towardsdatascience.com/predicting-stock-price-with-lstm-13af86a74944

In [ ]:
# Import Required Libraries
# Graph Functions
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,10) # Make the figures a bit bigger
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils
import pandas as pd
import os

## Import and Inspect Data

In [ ]:
display(os.getcwd())
df_ge = pd.read_csv('ge_stock.txt', engine='python')
df_ge.tail()

In [ ]:
from matplotlib import pyplot as plt
plt.figure()
plt.plot(df_ge["Open"])
plt.plot(df_ge["High"])
plt.plot(df_ge["Low"])
plt.plot(df_ge["Close"])
plt.title('GE stock price history')
plt.ylabel('Price (USD)')
plt.xlabel('Days')
plt.legend(['Open','High','Low','Close'], loc='upper left')
plt.show()

## Preprocess the Data
We need to normalize the data in order to improve convergence time.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

train_cols = ["Open","High","Low","Close","Volume"]
#train_cols = ["Open","High","Low","Volume"]
df_train, df_test = train_test_split(df_ge, train_size=0.8, test_size=0.2, shuffle=False)
print("Train and Test size", len(df_train), len(df_test))
# scale the feature MinMax, build array
x = df_train.loc[:,train_cols].values
min_max_scaler = MinMaxScaler()
x_train = min_max_scaler.fit_transform(x)
x_test = min_max_scaler.transform(df_test.loc[:,train_cols])

# Train scalar only on Y
stock_min_max = MinMaxScaler()
stock_min_max.fit_transform(np.reshape(df_train.loc[:, "Close"].values, (-1, 1)))

## Reshape the Data for Training
Our current dataset is not in a form suitable for training.

In [ ]:
import matplotlib.image as mpimg
img = mpimg.imread('stock_training.png')
plt.imshow(img)

In [ ]:
def build_timeseries(mat, y_col_index, TIME_STEPS):
    # y_col_index is the index of column that would act as output column
    # total number of time-series samples would be len(mat) - TIME_STEPS
    dim_0 = mat.shape[0] - TIME_STEPS
    dim_1 = mat.shape[1]
    x = np.zeros((dim_0, TIME_STEPS, dim_1))
    y = np.zeros((dim_0,))
    
    for i in range(dim_0):
        x[i] = mat[i:TIME_STEPS+i]
        y[i] = mat[TIME_STEPS+i, y_col_index]
    print("length of time-series i/o",x.shape,y.shape)
    return x, y


def trim_dataset(mat, batch_size):
    """
    trims dataset to a size that's divisible by BATCH_SIZE
    """
    no_of_rows_drop = mat.shape[0]%batch_size
    if(no_of_rows_drop > 0):
        return mat[:-no_of_rows_drop]
    else:
        return mat


In [ ]:
TIME_STEPS = 3
BATCH_SIZE = 32
x_t, y_t = build_timeseries(x_train, 3, TIME_STEPS)
x_t = trim_dataset(x_t, BATCH_SIZE)
y_t = trim_dataset(y_t, BATCH_SIZE)
x_temp, y_temp = build_timeseries(x_test, 3, TIME_STEPS)
x_val, x_test_t = np.split(trim_dataset(x_temp, BATCH_SIZE),2)
y_val, y_test_t = np.split(trim_dataset(y_temp, BATCH_SIZE),2)

## Model Creation
Time to create our Keras Model

In [ ]:
# Import functions from Keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(100, batch_input_shape=(BATCH_SIZE, TIME_STEPS, 
                                            x_t.shape[2]), 
                                            dropout=0.0, 
                                            recurrent_dropout=0.0))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(20,activation='relu'))
lstm_model.add(Dense(1,activation='linear'))
lstm_model.compile(loss='mean_squared_error', 
                   optimizer='sgd',
                   metrics=['mae'])

# Plot Model
The following code is to plot your created model. If you do not have certain outside libraries installed, the code may not run. This is Okay. You do not need this code to do the assignment.

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import SVG
#from keras.utils import model_to_dot
plot_model(lstm_model, show_shapes = True)

# Train and Plot Model Performance

In [ ]:
history = lstm_model.fit(x_t, y_t,
          batch_size=32, epochs=50, verbose=1,
          validation_data=(trim_dataset(x_val, BATCH_SIZE), 
                           trim_dataset(y_val, BATCH_SIZE)))

# Plot training & validation loss values
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')


# Generate Performance Metrics

In [ ]:
score = lstm_model.evaluate(x_t, y_t, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1] )

# Plot Validation Data against Actual

In [ ]:
predicted = lstm_model.predict(trim_dataset(x_val, BATCH_SIZE))
display(predicted)

In [ ]:
plt.figure()
plt.plot(stock_min_max.inverse_transform(predicted))
plt.plot(stock_min_max.inverse_transform(np.reshape(trim_dataset(y_val, BATCH_SIZE), (-1,1))))
plt.legend(['Estimated', 'Actual'], loc='upper left')

## Question 1: Changing Hyperparameters
Keras comes with pre-implemented loss functions and optimizers:

- Loss Function => https://keras.io/losses/
- Optimizer     => https://keras.io/optimizers/
         
Please pick two loss functions and two optimizers (for a total of four combinations) and compare their results using 10 epochs. Include a plot of the actual stock values and the predicted values for each combination and comment your observation.




## Fill in the table here

| Optimizer | Loss | Observations|
|-----------|------|-------------|
| Optimizer 1| Loss 1| Observation 1|
| Optimizer 1| Loss 2| Observation 2|
| Optimizer 2| Loss 1| Observation 3|
| Optimizer 2| Loss 2| Observation 4|


# Problem 2: Create Your Own Stock Prediction Algorithm
Follow similar steps to the above code, but for your own algorithm. Choose any stock and collect the data yourself (yahoo stocks is a good site).  

Create your own deep learning architecture (fully connected, CNN, or LSTM) to predict future stock prices for **Thirty days** and **1 Year into the future**. Note: the example above only predicted the next day’s stock value. You are not limited to the feature extraction, preprocessing, or validation process used in part 1. Be creative! 

Briefly describe why you chose the deep learning architecture you used, along with reasons for choosing the number of epochs, optimizer, activation functions, number of neurons in each layer, batch size, and the validation methodology.

Please turn in a .txt or .csv file that contains the data.